In [ ]:
from os import chdir, listdir, system

import pandas as pd
import pandas_datareader as pdr
import numpy as np
import matplotlib as mpl
import datetime

from requests import get
from lxml import html

from matplotlib import pylab as plt, dates as mpl_dates
mpl.rc('figure', max_open_warning=0)
pd.plotting.register_matplotlib_converters()

#import pandas_bokeh
#from quandl import get as qdl
#from pandas.plotting import register_matplotlib_converters
#from matplotlib import pyplot as plt, dates as mpl_dates
#register_matplotlib_converters()

import warnings
warnings.filterwarnings('ignore')

for x, y in enumerate(plt.style.available):
    print(f'{x} : {y}')

Num | Sty | Num | Sty | Num | Sty
:---:|:---:|:---:|:---:|:---:|:---:
0 | bmh | 1 | classic | 2 | dark_background
3 | fast | 4 | fivethirtyeight | 5 | ggplot
6 | grayscale | 7 | seaborn-bright | 8 | seaborn-colorblind
9 | seaborn-dark-palette | 10 | seaborn-dark | 11 | seaborn-darkgrid
12 | seaborn-deep | 13 | seaborn-muted | 14 | seaborn-notebook
15 | seaborn-paper | 16 | seaborn-pastel | 17 | seaborn-poster
18 | seaborn-talk | 19 | seaborn-ticks | 20 | seaborn-white
21 | seaborn-whitegrid | 22 | seaborn | 23 | Solarize_Light2
24 | tableau-colorblind10 | 25 | _classic_test

In [ ]:
%matplotlib inline
plt.style.use(plt.style.available[2])

In [ ]:
class Color():
    
    if 'system' not in globals():
        from os import system
    system('color')
    
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    END = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'

    BG_BLACK = '\033[99m'
    BG_WHITE = '\033[100m'
    BG_RED = '\033[101m'
    BG_GREEN = '\033[102m'
    BG_YELLOW = '\033[103m'
    BG_BLUE = '\033[104m'
    BG_PINK = '\033[105m'
    BG_CYAN = '\033[106m'
    BG_GREY = '\033[107m'
    pass

### Indices Financeiros / Papeis Aportados

In [ ]:
### Funções

def macd(Table, F:int=12, S:int=26, M:int=9):
    MACD = pd.DataFrame()
    MACD['MACD'] = Table.rolling(F).mean() - Table.rolling(S).mean()
    MACD['Signal'] = MACD['MACD'].rolling(M).mean()
    return MACD

def pivot(Table):
    valor_max = max(Table)
    valor_min = min(Table)

    PP = (valor_max + valor_min + Table[-2]) / 3
    # PP = (Data['High'][ref] + Data['Low'][ref] + Data['Close'][ref]) / 3

    R1 = (2 * PP) - valor_min
    # R1 = (2 * PP) - Data['Low'][ref]
    R2 = PP + (valor_max - valor_min)
    # R2 = PP + (Data['High'][ref] - Data['Low'][ref])
    R3 = valor_max + (2 * (PP - valor_min))
    # R3 = Data['High'][ref] + (2 * (PP - Data['Low'][ref]))

    S1 = (2 * PP) - valor_max
    # S1 = (2 * PP) - Data['High'][ref]
    S2 = PP - (valor_max - valor_min)
    # S2 = PP - (Data['High'][ref] - Data['Low'][ref])
    S3 = valor_min - (2 * (valor_max - PP))
    # S3 = Data['Low'][ref] - (2 * (Data['High'][ref] - PP))

    return np.array([[R1, R2, R3], [S1, S2, S3]], dtype='float64')

In [ ]:
tickers_index_base = ['^BVSP', 'USDBRL=X', 'GC=F', 'BTC-USD', 'CL=F']
#tickers_index_base = sorted(tickers_index_base)

with open('ticker.csv', 'r') as ticker_file:
    tickers_base = ticker_file.read().split('\n')
    tickers_base = sorted(tickers_base)
    tickers_base = list(filter(len, tickers_base))

In [ ]:
df_index = pd.DataFrame()
year = datetime.datetime.now().year
month = datetime.datetime.now().month
day = datetime.datetime.now().day
start_date = f'{year - 1:0.0f}-{month:0.0f}-{day:0.0f}'
remove_list = []

In [ ]:
for i in tickers_index_base:
    try:
        df_index[i] = pdr.DataReader(name=i, data_source='yahoo', start=start_date)['Adj Close']
        
        #df_index[i] = pdr.yahoo.daily.YahooDailyReader(symbols=i, start=start_date)['Adj Close']
    except Exception as err:
        print(err)
        print(f'{Color.RED}{i}{Color.END} nao foi localizado...')
        remove_list.append(i)

for i in remove_list:
    tickers_index_base.remove(i)

df_index.ffill(inplace=True)

In [ ]:
df_index.describe()

In [ ]:
index_log_returns = np.log(df_index / df_index.shift(1))
index_std_returns = index_log_returns.std()
index_var_returns = index_log_returns.var() * 250 ** 0.5
index_cov_returns = index_log_returns.cov() * 250
index_cor_returns = index_log_returns.corr()

In [ ]:
index_log_returns.tail()

In [ ]:
index_std_returns

In [ ]:
index_var_returns

In [ ]:
index_cov_returns

In [ ]:
index_cor_returns

In [ ]:
df_index.tail(10)

In [ ]:
(df_index / df_index.iloc[0] * 100).plot(figsize=(20,10))

plt.tight_layout()

In [ ]:
df_base = df_index.copy()
remove_list = []

In [ ]:
for i in tickers_base:
    try:
        df_base[i] = pdr.DataReader(name= i + '.SA', data_source='yahoo', start=start_date)['Adj Close']
    except Exception as err:
        print(err)
        print(f'{i} nao esta disponivel ou não existe...')
        remove_list.append(i)

for i in remove_list:
    try:
        tickers_index_base.remove(i)
    except:
        pass

df_base.ffill(inplace=True)

In [ ]:
df_base.describe().to_excel('describe.xlsx')
df_base.describe()

In [ ]:
df_base.head(5)

In [ ]:
df_base.tail(5)

In [ ]:
base_log_returns = np.log(df_base / df_base.shift(1))
base_std_returns = base_log_returns.std()
base_var_returns = base_log_returns.var() * 250 ** 0.5
base_cov_returns = base_log_returns.cov() * 250
base_cor_returns = base_log_returns.corr()

In [ ]:
base_log_returns.to_excel('base_log_returns.xlsx')
base_cov_returns.to_excel('base_cov_returns.xlsx')
base_cor_returns.to_excel('base_cor_returns.xlsx')
base_log_returns.tail()

In [ ]:
base_std_returns * 100

In [ ]:
base_var_returns * 100

In [ ]:
base_cov_returns

In [ ]:
base_cor_returns

In [ ]:
#(df_base / df_base.iloc[0] * 100).plot(figsize=(20,10), color=['black', 'grey', 'gold', 'c', 'y', 'g', 'b', 'darkgreen', 'm', 'red', 'pink'], lw=1)
(df_base / df_base.iloc[0] * 100).plot(figsize=(20,10), lw=1)

plt.title('Gráfico Normalizado para Comparação de Valorização Geral')

plt.tight_layout()
plt.savefig('geral_normalizado_em_100.jpg')

t = tickers_base[-1]

last_value = df_base[t].iloc[-1]

min_val = df_base[t].iloc[-250:].min()
min_val_index = df_base[t].iloc[-250:].idxmin()
min_val_mean = df_base[t].loc[min_val_index:].mean()

max_val = df_base[t].iloc[-250:].max()
max_val_index = df_base[t].iloc[-250:].idxmax()
max_val_mean = df_base[t].loc[max_val_index:].mean()

slope = min_val / last_value
corredor_inicio = max_val * slope
corredor_fim = max_val * (1 + slope)

[
    [last_value, slope, corredor_inicio, corredor_fim],
    [min_val, min_val_index, min_val_mean],
    [max_val, max_val_index, max_val_mean]
    ]

data = min_val_index.date() - df_base[t].index[-1].date()
data.days

In [ ]:
periodo = -250
fig, ax = plt.subplots(ncols=3, nrows=len(tickers_base), figsize=(30, len(tickers_base) * 5))

for row, item in enumerate(tickers_base):
    # Normalização
    try:
        analisys = tickers_index_base.copy()
        analisys.append(tickers_base[row])
        ax[row, 0].plot(df_base.iloc[periodo:][analisys] / df_base[analisys].iloc[periodo] * 100, lw=1)
        #(df_base / df_base.iloc[0] * 100).plot(figsize=(20,10), color=['black', 'grey', 'gold', 'c', 'y', 'g', 'b', 'darkgreen', 'm', 'red', 'pink'], lw=1)
        #ax[row, 0].set_color(['grey', 'grey', 'grey', 'grey', 'black'])
        ax[row, 0].legend(analisys)
        ax[row, 0].axhline(100, color='r', lw=1, ls=':', alpha=1)
        pass
    except Exception as err:
        print(err)
        pass
    # Analise Tecnica I
    try:
        # Linha de Tendencia
        t = tickers_base[row]

        last_value = df_base[t].iloc[-1]

        min_val = df_base[t].iloc[periodo:].min()
        min_val_index = df_base[t].iloc[periodo:].idxmin()

        max_val = df_base[t].loc[min_val_index:].max()
        max_val_index = df_base[t].loc[min_val_index:].idxmax()
        
        mean_val = df_base[t].loc[min_val_index:].mean()

        # data = min_val_index.date() - df_base[t].index[-1].date()
        # per = -1 * data.days

        # slope = min_val / mean_val
        # corredor_inicio = min_val * (1 + slope)
        # corredor_fim = (max_val * slope) + corredor_inicio #*  (1 + slope)

        ax[row, 1].plot(
            [df_base.index[periodo], df_base.index[-1]],
            [df_base.iloc[periodo][t], np.mean(df_base.iloc[periodo:][t])],
            color='pink', lw=2, ls=':', label='Tendencia', alpha=0.5)
        
        ax[row, 1].plot(
            [min_val_index, df_base.index[-1]],
            [min_val, mean_val],
            color='pink', lw=2, ls=':', alpha=0.5)
        
        # ax[row, 1].plot(
        #     [min_val_index, df_base.index[-1]],
        #     [corredor_inicio, corredor_fim],
        #     color='pink', lw=2, ls=':', alpha=0.5)

        # Menor valor dos ultimos 22 dias
        menor_22_dias = min(df_base.iloc[-22:-1][tickers_base[row]])
        ax[row, 1].text(df_base.index[periodo//2], menor_22_dias * 1.02, f'Menor Valor 22 Dias : {menor_22_dias:0.2f}', va='center', ha='center', color='darkred', bbox=dict(facecolor='white', alpha=0.5))
        ax[row, 1].axhline(menor_22_dias, color='gold', lw=1, ls=':')#, label=f'Min V. 22: {menor_22_dias:0.2f}', alpha=1)
        ax[row, 1].axvline(df_base.index[-22], color='gold', lw=1, ls=':', alpha=1)
        
        # Média Movel Aritimética 21/72 dias
        mma = pd.DataFrame()
        mma['21'] = df_base[tickers_base[row]].rolling(21).mean()
        mma['72'] = df_base[tickers_base[row]].rolling(72).mean()
        ax[row, 1].plot(mma['21'].iloc[periodo:], lw=1, label='MMA 21', color='white', alpha=0.5)
        ax[row, 1].plot(mma['72'].iloc[periodo:], lw=1, label='MMA 72', color='yellow', alpha=0.5)
        
        # Pivot
        piv_array = pivot(df_base.iloc[-22:][tickers_base[row]])
        piv_per = [df_base.index[-22], df_base.index[-22], df_base.index[-22]]
        ax[row, 1].plot(piv_per[0], piv_array.mean(), '*', color='gold')
        ax[row, 1].plot(piv_per, piv_array[0], '*', color='c', label='Res.')
        ax[row, 1].plot(piv_per, piv_array[1], '*', color='r', label='Sup.')

        # Principal
        last_price = df_base.iloc[-1][tickers_base[row]]
        lowest_price = min(df_base.iloc[periodo:][tickers_base[row]])
        higher_price = max(df_base.iloc[periodo:][tickers_base[row]])
        ax[row, 1].text(df_base.index[periodo//2], higher_price, f'{higher_price:0.2f}', va='top', ha='center', color='white', bbox=dict(facecolor='green', alpha=0.5))
        ax[row, 1].text(df_base.index[periodo//2], lowest_price, f'{lowest_price:0.2f}', va='bottom', ha='center', color='white', bbox=dict(facecolor='red', alpha=0.5))
        ax[row, 1].text(df_base.index[-1], last_price * 1.02, f'{last_price:0.2f}', va='bottom', ha='left', color='gold', bbox=dict(facecolor='blue', alpha=0.5))
        ax[row, 1].plot(df_base.iloc[periodo:][tickers_base[row]], lw=1.5, label=tickers_base[row])
        ax[row, 1].legend()
        pass
    except Exception as er:
        print(er)
        pass
    # MACD / Signal
    try:
        mac = macd(df_base[tickers_base[row]].iloc[periodo:])
        # ax[row, 2].bar(base_log_returns.index[periodo:], base_log_returns[item].iloc[periodo:] * 10, color='silver', alpha=0.5, label='Retorno Log.')
        ax[row, 2].plot(mac['MACD'].iloc[periodo:], color='c', lw=2, label='MACD')
        ax[row, 2].plot(mac['Signal'].iloc[periodo:], color='r', lw=1, label='Signal')
        ax[row, 2].axhline(0, color='r', lw=1, ls=':', alpha=0.5)
        ax[row, 2].axvline(mac['Signal'].index[-22], color='r', lw=1, ls=':', alpha=0.5)
        intersections = np.argwhere(np.diff(np.sign(mac['MACD'] - mac['Signal']))).flatten()
        ax[row, 2].plot(mac['Signal'].iloc[intersections], '*', color='y')
        ax[row, 2].legend()
        pass
    except:
        pass

plt.tight_layout()
fig.savefig('analise_geral_individual.jpg')

## Analise Fundamentalista

In [ ]:
# Funções

def Fund_Data(ticker:str, url:str):
    agent = {'User-Agent':'Mozilla/5.0'}
    page = f'{url+ticker}'
    content = get(page, headers=agent).text
    return content #.encode('utf-8')

def Preco_Justo(VPA:float, LPA:float, COTACAO:float):
    if VPA > 0 and LPA > 0:
        preco_justo = (22 * LPA * VPA) ** 0.5
        fator = ((preco_justo - COTACAO) - 1) * 100
        diferenca = preco_justo - COTACAO
        return (preco_justo, fator, diferenca)
    else:
        return (0 , 0 , 0)

In [ ]:
_urlbase = 'http://www.fundamentus.com.br/detalhes.php?papel='

In [ ]:
data_fundamentalist = {}

for t in tickers_base:
    try:
        data = Fund_Data(t, _urlbase)
        df_page = pd.read_html(data.encode('iso-8859-1'), thousands='.')

        data_fundamentalist[t] = {
            'Empresa' : df_page[0].iloc[2,1],
            'Setor' : df_page[0].iloc[3,1],
            'Sub-Setor' : df_page[0].iloc[4,1],
            'Cotacao' : float(df_page[0].iloc[0,3].replace(',', '.')),
            'Num_Acoes' : float(df_page[1].iloc[1,3].replace(',', '.')),
            'LPA' : float(df_page[2].iloc[1,5].replace(',', '.')),
            'VPA' : float(df_page[2].iloc[2,5].replace(',', '.'))
            }
    except:
        pass

In [ ]:
fig, ax = plt.subplots(nrows=int(np.ceil(len(tickers_base) / 2)), ncols=2, figsize=(20, (len(tickers_base) / 2) * 1.75))

r, c = 0, 0

for key in data_fundamentalist.keys():
    cota = data_fundamentalist[key]['Cotacao']
    v = Preco_Justo(data_fundamentalist[key]['VPA'], data_fundamentalist[key]['LPA'], cota)
    # print(f'{key} : PrecoJusto = {v[0]:0.3f} / Fator = {v[1]:0.3f}')
    # print(f'{key} : Diferença = {v[2]:0.3f}')
    vl = [data_fundamentalist[key]['VPA'], data_fundamentalist[key]['LPA']]
    if vl[0] > 0 and vl[1] > 0:
        cor_1 = 'grey'
    else:
        cor_1 = 'darkred'
    
    if v[0] > cota:
        cor_2 = 'gold'
    else:
        cor_2 = 'white'

    ax[r, c].bar(key, data_fundamentalist[key]['Cotacao'], label='Cotação', color=cor_1)
    ax[r, c].bar(key, v[0], alpha=0.75, label='Preço Justo', color=cor_2)

    #ax[r, c].legend()

    ax[r, c].text(-0.175, v[0], f'Preço Justo : {v[0]:0.2f}', va='center', ha='center', color='black', bbox=dict(facecolor='white', alpha=0.75))
    ax[r, c].text(0.175, cota, f'Cotação : {cota:0.2f}', va='center', ha='center', color='black', bbox=dict(facecolor='white', alpha=0.75))
    ax[r, c].axhline(v[0], ls=':', lw=0.5, color='red')

    if c == 0:
        c = 1
    else:
        c = 0
        r += 1
#fig.suptitle('Análise Preço Justo')
#fig.legend()
plt.tight_layout()
fig.savefig('fund_graph.jpg')